In [1]:
from pathlib import Path
from anomalib.data import Folder
from anomalib.engine import Engine
from anomalib.models import Padim, Patchcore
import matplotlib.pyplot as plt
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping
from torchvision import transforms
from anomalib.pre_processing import PreProcessor
from torchvision.transforms.v2 import Compose, Resize, Normalize, CenterCrop, InterpolationMode, ToTensor
from anomalib.models import Patchcore
from anomalib.metrics import AUROC, F1Score, Evaluator, AUPR, precision_recall_curve


transform = Compose([
    CenterCrop(900),
    Resize(size=[256, 256], interpolation=InterpolationMode.BILINEAR, antialias=True),
    Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    #ToTensor(),
])
pre_processor = PreProcessor(transform=transform)

# Create metrics
metrics = [
    AUROC(fields=["pred_score", "gt_label"]),
    AUPR(fields=["pred_score", "gt_label"]),
    F1Score(fields=["pred_label", "gt_label"]),
]

# Create evaluator with metrics
evaluator = Evaluator(test_metrics=metrics)

#early_stopping = EarlyStopping(monitor="AUROC", patience=3, mode="max")

# Step 2: Setup the datamodule with the transform
datamodule = Folder(
    name="Images",
    root=Path("Images"),
    normal_dir="normal",         # Subfolder containing normal images
    abnormal_dir="anomaly",      # Subfolder containing anomalous images (Needed for Padim)
    mask_dir=None,            
    normal_split_ratio=0.2,
    train_batch_size=8,       # 1 for EfficientAd and 32 for Padim
    eval_batch_size=8,
    num_workers=2,
)

# Step 3: Define the model
#model = Padim(
#    backbone="resnet50",             # Feature extraction backbone
#    layers=["layer3", "layer4", "layer4"],  # Layers to extract features from
#    pre_trained=True,
#    n_features=550,
#    evaluator=evaluator,
#    pre_processor=pre_processor,
#)

# Step 3: Define the model
model = Patchcore(
    backbone="resnet50",             # Feature extraction backbone
    layers=["layer3", "layer4"],  # Layers to extract features from
    pre_trained=True,
    num_neighbors=5,
    evaluator=evaluator,
    pre_processor=pre_processor,
)

# Step 4: Initialize training engine
engine = Engine(
    #callbacks=[early_stopping],  # Wrap early_stopping in a list
    accelerator="gpu",
    devices=1,
)

# Step 5: Train the model
engine.fit(
    model=model,
    datamodule=datamodule,
)

engine.test(model, datamodule=datamodule)


INFO:numexpr.utils:NumExpr defaulting to 4 threads.
c:\Users\Ditle\anaconda3\Lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(
c:\Users\Ditle\anaconda3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
INFO:anomalib.models.components.base.anomalib_module:Initializing Patchcore model.
c:\Users\Ditle\anaconda3\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'pre_processor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['pre_processor'])`.
c:\Users\Ditle\anaconda3\Lib\site-packages\lightning\pyt

Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 8.62 GiB is allocated by PyTorch, and 578.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)